# Signal Spaces

## Basic types
This module define the types of the signal spaces, which is the types of the elments in the signal module. The type here behaves like configuration.

- `SigNum`: the type that will be used in 0-sig, 1-sig and 3-sig

In [1]:
{-# LANGUAGE  ConstraintKinds #-}

module Signal.Types
(SigNum
) where

type SigNum a = (Num a, Ord a, Show a)

## Utils
Utility functions used in the signal module.

- `ispower2::Int->Bool`: check if the input `int` is a power of 2
- `linespace::(Fractiona a, Ord a, Integral n)=>a->a->n->[a]`: function in the form of `f(a,b,n)` generates the sequence of equal spacing fractional numbers in interval `[a,b]` with length of `n`.
- `windowed :: Int->[a]->[[a]]`: give rise to sliding windows on the input list of type `a`.


In [3]:
module Signal.Utils
(ispower2
,linespace
,windowed
) where

import Data.Bits

ispower2::Int->Bool
ispower2 n
    | n<=0 = False
    | otherwise = (n .&. (n-1))==0

linespace :: (Fractional a, Ord a, Integral n)=>a->a->n->[a]
linespace a1 a2 _
    | a2<=a1 = []
linespace a b n
    | n<=0 = []
    | n==1 = [a, b]
linespace a b n = [let dt=(b-a)/fromIntegral(n-1) in a+dt*fromIntegral i | i<-[0..(n-1)]]

-- windowed on list, returning windows with n elements  each
windowed :: Int->[a]->[[a]]
windowed n x
    | n<=1 = [x]
windowed _ [] = []
windowed n (x:xs)
    | length(take n (x:xs))<n = [] -- `take n` makes sure it works on infinite list
    | otherwise = w:windowed n xs
    where w=take n (x:xs)
-- without guarantee each window has n element, then the following works:
-- windowed n x = Data.List.transpose $ take n $ tails x
-- or
-- windowed n x = map (tak n) $ tails x

## Padding

In [5]:
{-# LANGUAGE  ConstraintKinds, RankNTypes #-}

module Signal.Padding
(Padding(..)
,Extensions(..)
,PaddingMethod(ZERO,PERIODIC,SYMETRIC)
) where

import Signal.Types

data Extensions a = Extensions{left::[a]
                              ,right::[a]}
instance (Show a)=>Show (Extensions a) where
    show Extensions{left=l,right=r} = "["++ show l ++ "," ++ show r ++ "]" ++ "\n"

class Padding f where
    pad::SigNum a=>f->(Int->[a]->Extensions a)

data PaddingMethod=ZERO|PERIODIC|SYMETRIC deriving(Eq)

type GetExtensions a = SigNum a=>Int->[a]->Extensions a

zero::GetExtensions a
zero n a = Extensions{left=replicate n 0, right=replicate n 0}

symetric::GetExtensions a
symetric n a = Extensions{left=lr,right=rr} where
    lr = reverse $ take n a
    rr = take n $ reverse a

periodic::GetExtensions a
periodic n a = Extensions{left=lr,right=rr} where
    lr = reverse $ take n  $ reverse a
    rr = take n a

instance Padding PaddingMethod where
    pad ZERO = zero
    pad PERIODIC = periodic
    pad SYMETRIC = symetric

In [6]:
{-# LANGUAGE  TypeFamilies #-}

module Signal.Model
(Sig_0
,Sig_1
,blocks
,Triple
,Sig_3
,oblocks
,Sig
,SigF
) where

import qualified Signal.Utils as Utils
import Signal.Types
import Signal.Padding

type family SigF a where
  SigF a = [a]

type Sig_0 a = SigNum a=>SigF a
type Sig_1 a = SigNum a=>SigF [a]

--type Triple::Type->Type->Type
type Triple a  = ([a],[a],[a])

type Sig_3 a = SigNum a=>SigF (Triple a)

blocks::forall a.SigNum a=>Int->Sig_0 a->Sig_1 a
blocks n x
    | not $ Utils.ispower2 n = error "illegal size of blocks"
    | n<=0 = [x]
blocks _ [] = []
blocks n xs = take n xs:blocks n (drop n xs)

oblocksInner::Int->Sig_1 a->Sig_3 a
oblocksInner n (x:xs)
    | length (take 3 (x : xs))  < 3 = [] -- `take 3` intended to make it work for infinite sequence
    | otherwise = (drop (length x-n) $ head w, w!!1, take n $ last w):oblocksInner n xs
    where
        w=take 3 (x:xs)

oblocks::SigNum a=>PaddingMethod->Int->Sig_1 a->Sig_3 a
oblocks pm n xs
    | (n > length xs) || not (Utils.ispower2 n) = error "illegal size of blocks"
    | pm == PERIODIC = oblocksInner n (last xs : xs ++ [head xs])
    | pm == ZERO = oblocksInner n (replicate n 0:xs ++ [replicate n 0])
    | pm == SYMETRIC = oblocksInner n (reverse (head xs):xs++[reverse $ last xs])
    | otherwise = error "unkown method"

class Sig s where
    transform::(a->a)->s a->s a

instance Sig [] where
    transform=fmap

In [7]:
import Signal.Model

In [8]:
module Signal.Algorithms
(decimation
) where

decimation::Int->[a]->[a]
decimation n x
    | n<2 = x
decimation _ [] = []
decimation n (x:xs) = x:decimation n (drop (n-1) xs)